In [1]:
import isaid_helpers
import pandas as pd

In [2]:
pd.read_csv(isaid_helpers.f_graphable_doi).head()

,doi,name,url,publisher,date_qualifier,year_published,entity_type,journal,description,event
0,10.3133/ofr2000247,Fish community structure in relation to enviro...,http://dx.doi.org/10.3133/OFR2000247,US Geological Survey,2021-03-31T06:34:09.293702,2000.0,CreativeWork,USGS Open-File Report,NaN,NaN
1,10.1002/eap.2243,Resistance and resilience of pelagic and litto...,http://dx.doi.org/10.1002/EAP.2243,Wiley,2021-03-31T06:37:06.414581,2021.0,CreativeWork,Ecological Applications,NaN,NaN
2,10.3133/70039590,National Cartographic Information Center Newsl...,http://dx.doi.org/10.3133/70039590,US Geological Survey,2021-03-31T22:08:47.934746,1975.0,CreativeWork,USGS Newsletter,NaN,NaN
3,10.3133/wsp2163,"Ground-water levels in the United States, 1972...",http://dx.doi.org/10.3133/WSP2163,US Geological Survey,2021-03-31T22:25:31.855010,1977.0,CreativeWork,USGS [],NaN,NaN
4,10.3133/ofr83254,Water-resources investigations of the U.S. Geo...,http://dx.doi.org/10.3133/OFR83254,US Geological Survey,2021-03-31T22:28:57.198706,1983.0,CreativeWork,USGS Open-File Report,NaN,NaN


In [3]:
%%time
with isaid_helpers.graph_driver.session(database=isaid_helpers.graphdb) as session:
    session.run("""
        LOAD CSV WITH HEADERS FROM '%(source_path)s/%(source_file)s' AS row
        WITH row
            WHERE row.entity_type = "CreativeWork"
                MERGE (cw:CreativeWork {doi: row.doi})
                ON CREATE
                    SET cw.name = row.name,
                    cw.url = row.url,
                    cw.publisher = row.publisher,
                    cw.year_published = row.year_published,
                    cw.journal = row.journal,
                    cw.description = row.description,
                    cw.event = row.event
    """ % {
        "source_path": isaid_helpers.local_cache_path,
        "source_file": isaid_helpers.f_graphable_doi
    })

CPU times: user 1.33 ms, sys: 1.36 ms, total: 2.69 ms
Wall time: 1.58 s


In [4]:
%%time
with isaid_helpers.graph_driver.session(database=isaid_helpers.graphdb) as session:
    session.run("""
        LOAD CSV WITH HEADERS FROM '%(source_path)s/%(source_file)s' AS row
        WITH row
            WHERE row.entity_type = "Dataset"
                MERGE (cw:Dataset {doi: row.doi})
                ON CREATE
                    SET cw.name = row.name,
                    cw.url = row.url,
                    cw.publisher = row.publisher,
                    cw.year_published = row.year_published,
                    cw.journal = row.journal,
                    cw.description = row.description,
                    cw.event = row.event
    """ % {
        "source_path": isaid_helpers.local_cache_path,
        "source_file": isaid_helpers.f_graphable_doi
    })

CPU times: user 1.34 ms, sys: 1.74 ms, total: 3.07 ms
Wall time: 5.79 s


In [5]:
%%time
with isaid_helpers.graph_driver.session(database=isaid_helpers.graphdb) as session:
    session.run("""
        LOAD CSV WITH HEADERS FROM '%(source_path)s/%(source_file)s' AS row
        WITH row
            WHERE NOT row.event IS NULL
                MERGE (e:Event {name: row.event})
                ON CREATE
                    SET e.original_source = "DOI Record",
                    e.reference = row.url
    """ % {
        "source_path": isaid_helpers.local_cache_path,
        "source_file": isaid_helpers.f_graphable_doi
    })

CPU times: user 1.22 ms, sys: 2.57 ms, total: 3.79 ms
Wall time: 192 ms


In [6]:
%%time
with isaid_helpers.graph_driver.session(database=isaid_helpers.graphdb) as session:
    session.run("""
        LOAD CSV WITH HEADERS FROM '%(source_path)s/%(source_file)s' AS row
        WITH row
            WHERE NOT row.journal IS NULL
                MERGE (j:Journal {name: row.journal})
                ON CREATE
                    SET j.original_source = "DOI Record"
    """ % {
        "source_path": isaid_helpers.local_cache_path,
        "source_file": isaid_helpers.f_graphable_doi
    })

CPU times: user 1.39 ms, sys: 1.88 ms, total: 3.28 ms
Wall time: 10.4 s


In [7]:
%%time
with isaid_helpers.graph_driver.session(database=isaid_helpers.graphdb) as session:
    session.run("""
        LOAD CSV WITH HEADERS FROM '%(source_path)s/%(source_file)s' AS row
        WITH row
            WHERE NOT row.event IS NULL
                MATCH (cw:CreativeWork {doi: row.doi})
                WITH cw, row
                    MATCH (e:Event {name: row.event})
                    MERGE (cw)-[cwe:PRODUCT_OF]->(e)
                        SET cwe.reference = row.url,
                        cwe.date_qualifier = row.year_published
    """ % {
        "source_path": isaid_helpers.local_cache_path,
        "source_file": isaid_helpers.f_graphable_doi
    })

CPU times: user 1.22 ms, sys: 2.72 ms, total: 3.95 ms
Wall time: 740 ms


In [8]:
%%time
with isaid_helpers.graph_driver.session(database=isaid_helpers.graphdb) as session:
    session.run("""
        LOAD CSV WITH HEADERS FROM '%(source_path)s/%(source_file)s' AS row
        WITH row
            WHERE NOT row.journal IS NULL
                MATCH (cw:CreativeWork {doi: row.doi})
                WITH cw, row
                    MATCH (j:Journal {name: row.journal})
                    MERGE (cw)-[cwj:PUBLISHED_IN]->(j)
                        SET cwj.reference = row.url,
                        cwj.date_qualifier = row.year_published
    """ % {
        "source_path": isaid_helpers.local_cache_path,
        "source_file": isaid_helpers.f_graphable_doi
    })

CPU times: user 1.69 ms, sys: 1.53 ms, total: 3.22 ms
Wall time: 17.6 s


In [9]:
pd.read_csv(isaid_helpers.f_graphable_doi_contacts).head()

,doi,reference,date_qualifier,orcid,sequence,rel_type,entity_type,name
0,10.3133/ofr2000247,http://dx.doi.org/10.3133/OFR2000247,2000,0000-0001-6702-4531,additional,AUTHOR_OF,Person,Larry R. Brown
1,10.1002/eap.2243,http://dx.doi.org/10.1002/EAP.2243,2021,0000-0001-6702-4531,additional,AUTHOR_OF,Person,Larry Brown
2,10.3133/sir20085004,http://dx.doi.org/10.3133/SIR20085004,2008,0000-0002-6060-9729,first,AUTHOR_OF,Person,Nathan Wood
3,10.3133/sir20085236,http://dx.doi.org/10.3133/SIR20085236,2008,0000-0002-4381-0746,first,AUTHOR_OF,Person,Stephen B. Gingerich
4,10.3133/fs20063139,http://dx.doi.org/10.3133/FS20063139,2006,0000-0002-6214-6182,first,AUTHOR_OF,Person,Cynthia A. Gardner


In [10]:
%%time
with isaid_helpers.graph_driver.session(database=isaid_helpers.graphdb) as session:
    session.run("""
        LOAD CSV WITH HEADERS FROM '%(source_path)s/%(source_file)s' AS row
        WITH row
            WHERE row.rel_type = "AUTHOR_OF"
                MATCH (cw:CreativeWork {doi: row.doi})
                WITH cw, row
                    MATCH (p:Person {orcid: row.orcid})
                    MERGE (p)-[a:AUTHOR_OF]->(cw)
                    ON CREATE
                        SET a.reference = row.reference,
                        a.date_qualifier = row.date_qualifier,
                        a.sequence = row.sequence
    """ % {
        "source_path": isaid_helpers.local_cache_path,
        "source_file": isaid_helpers.f_graphable_doi_contacts
    })

CPU times: user 3.65 ms, sys: 2.6 ms, total: 6.24 ms
Wall time: 2min 34s


In [11]:
%%time
with isaid_helpers.graph_driver.session(database=isaid_helpers.graphdb) as session:
    session.run("""
        LOAD CSV WITH HEADERS FROM '%(source_path)s/%(source_file)s' AS row
        WITH row
            WHERE row.rel_type = "EDITOR_OF"
                MATCH (cw:CreativeWork {doi: row.doi})
                WITH cw, row
                    MATCH (p:Person {orcid: row.orcid})
                    MERGE (p)-[a:EDITOR_OF]->(cw)
                    ON CREATE
                        SET a.reference = row.reference,
                        a.date_qualifier = row.date_qualifier,
                        a.sequence = row.sequence
    """ % {
        "source_path": isaid_helpers.local_cache_path,
        "source_file": isaid_helpers.f_graphable_doi_contacts
    })

CPU times: user 1.13 ms, sys: 1.9 ms, total: 3.04 ms
Wall time: 2.11 s


In [12]:
pd.read_csv(isaid_helpers.f_graphable_doi_funders).head()

,doi,reference,date_qualifier,name,rel_type,entity_type,funder_doi,funder_award
0,10.1002/eap.2243,http://dx.doi.org/10.1002/EAP.2243,2021,Department of Water Resources,FUNDER_OF,Organization,10.13039/100004813,NaN
1,10.1002/eap.2243,http://dx.doi.org/10.1002/EAP.2243,2021,Bureau of Reclamation,FUNDER_OF,Organization,10.13039/100006450,NaN
2,10.1016/j.jglr.2013.12.011,http://dx.doi.org/10.1016/J.JGLR.2013.12.011,2014,U.S. Environmental Protection Agency,FUNDER_OF,Organization,10.13039/100000139,NaN
3,10.1080/10871209.2017.1324069,http://dx.doi.org/10.1080/10871209.2017.1324069,2017,"South Dakota Game, Fish and Parks",FUNDER_OF,Organization,10.13039/100014298,T-59-R1
4,10.1016/j.biocon.2016.05.015,http://dx.doi.org/10.1016/J.BIOCON.2016.05.015,2017,U.S. Forest Service and the Ecological Society...,FUNDER_OF,Organization,NaN,12-CA-11221633-096


In [13]:
%%time
with isaid_helpers.graph_driver.session(database=isaid_helpers.graphdb) as session:
    session.run("""
        LOAD CSV WITH HEADERS FROM '%(source_path)s/%(source_file)s' AS row
        WITH row
            MERGE (o:Organization {name: row.name})
            ON CREATE
                SET o.category = "Funding Organization",
                o.original_source = "DOI Record",
                o.doi = row.funder_doi

                WITH o, row
                    MATCH (cw:CreativeWork {doi: row.doi})
                    MERGE (cw)-[fb:FUNDED_BY]->(o)
                    ON CREATE
                        SET fb.reference = row.reference,
                        fb.date_qualifier = row.date_qualifier,
                        fb.funder_award = row.funder_award
    """ % {
        "source_path": isaid_helpers.local_cache_path,
        "source_file": isaid_helpers.f_graphable_doi_contacts
    })

CPU times: user 1.4 ms, sys: 1.69 ms, total: 3.09 ms
Wall time: 709 ms


In [14]:
pd.read_csv(isaid_helpers.f_graphable_doi_terms).head()

,doi,reference,date_qualifier,name,rel_type,entity_type
0,10.1002/eap.2243,http://dx.doi.org/10.1002/EAP.2243,2021,Ecology,ADDRESSES_SUBJECT,UndefinedSubjectMatter
1,10.1016/j.jglr.2013.12.011,http://dx.doi.org/10.1016/J.JGLR.2013.12.011,2014,Ecology,ADDRESSES_SUBJECT,UndefinedSubjectMatter
2,10.1016/j.jglr.2013.12.011,http://dx.doi.org/10.1016/J.JGLR.2013.12.011,2014,Aquatic Science,ADDRESSES_SUBJECT,UndefinedSubjectMatter
3,10.1016/j.jglr.2013.12.011,http://dx.doi.org/10.1016/J.JGLR.2013.12.011,2014,"Ecology, Evolution, Behavior and Systematics",ADDRESSES_SUBJECT,UndefinedSubjectMatter
4,10.1111/conl.12095,http://dx.doi.org/10.1111/CONL.12095,2014,Ecology,ADDRESSES_SUBJECT,UndefinedSubjectMatter


In [15]:
%%time
with isaid_helpers.graph_driver.session(database=isaid_helpers.graphdb) as session:
    session.run("""
        LOAD CSV WITH HEADERS FROM '%(source_path)s/%(source_file)s' AS row
        WITH row
            MERGE (t:UndefinedSubjectMatter {name: row.name})
            ON CREATE
                SET t.original_source = "DOI Record"

                WITH t, row
                    MATCH (cw:CreativeWork {doi: row.doi})
                    MERGE (cw)-[s:ADDRESSES_SUBJECT]->(t)
                    ON CREATE
                        SET s.reference = row.reference,
                        s.date_qualifier = row.date_qualifier
    """ % {
        "source_path": isaid_helpers.local_cache_path,
        "source_file": isaid_helpers.f_graphable_doi_terms
    })

CPU times: user 1.23 ms, sys: 1.38 ms, total: 2.61 ms
Wall time: 380 ms
